In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [41]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/game/train.csv")
games = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/game/games.csv")
turns = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/game/turns.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/game/test.csv")
sub = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/game/sample_submission.csv')

In [42]:
print("train shape:", train.shape)
print("test shape:", test.shape)
print("turns shape:", turns.shape)
print("games shape:", games.shape)
print("submission shape:", sub.shape)

train shape: (100820, 4)
test shape: (44726, 4)
turns shape: (2005498, 9)
games shape: (72773, 12)
submission shape: (22363, 2)


# train+test csv

In [43]:
train.head()

,game_id,nickname,score,rating
0,1,BetterBot,335,1637
1,1,stevy,429,1500
2,3,davidavid,440,1811
3,3,BetterBot,318,2071
4,4,Inandoutworker,119,1473


In [44]:
# Creating dataframe of the bot information for each of the games
bot_names =["BetterBot", "STEEBot", "HastyBot"]
bot_df = train[["game_id", "nickname", "score", "rating"]].copy()
bot_df['bot_name'] = bot_df['nickname'].apply(lambda x: x if x in bot_names else np.nan)
bot_df = bot_df[["game_id", "score", "rating", "bot_name"]].dropna(subset=["bot_name"])
bot_df.columns = ["game_id", "bot_score", "bot_rating", "bot_name"]

In [45]:
full_df = train[~train['nickname'].isin(bot_names)] #take out the bots
full_df = full_df.merge(bot_df, on="game_id") #add in bot information
full_df = full_df.merge(games, on="game_id") # add in game information

In [46]:
full_df.head()

,game_id,nickname,score,rating,bot_score,bot_rating,bot_name,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
0,1,stevy,429,1500,335,1637,BetterBot,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274
1,3,davidavid,440,1811,318,2071,BetterBot,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262
2,4,Inandoutworker,119,1473,478,1936,BetterBot,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141
3,5,stevy,325,1500,427,1844,STEEBot,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722
4,6,HivinD,378,2029,427,2143,STEEBot,HivinD,regular,STANDARD,0,2022-08-21 14:56:35,CSW21,900,0,RATED,1,426.950541


In [27]:
full_df['increment_seconds'].value_counts()

0     49056
10      544
1       435
5       184
60      100
7        45
3        34
15        7
59        4
30        1
Name: increment_seconds, dtype: int64

In [28]:
full_df.shape

(50410, 18)

In [30]:
full_df.drop(['increment_seconds','created_at'],axis=1,inplace=True)

In [31]:
full_df.shape

(50410, 16)

In [32]:
full_df['first'] = np.where(full_df['nickname'] == full_df['first'],1,0)

In [33]:
full_df.head()

,game_id,nickname,score,rating,bot_score,bot_rating,bot_name,first,time_control_name,game_end_reason,winner,lexicon,initial_time_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
0,1,stevy,429,1500,335,1637,BetterBot,0,regular,STANDARD,1,NWL20,1200,CASUAL,1,674.844274
1,3,davidavid,440,1811,318,2071,BetterBot,0,regular,STANDARD,1,CSW21,900,RATED,5,492.268262
2,4,Inandoutworker,119,1473,478,1936,BetterBot,0,regular,RESIGNED,0,CSW21,3600,CASUAL,1,350.861141
3,5,stevy,325,1500,427,1844,STEEBot,0,regular,STANDARD,0,NWL20,1200,CASUAL,1,642.688722
4,6,HivinD,378,2029,427,2143,STEEBot,1,regular,STANDARD,0,CSW21,900,RATED,1,426.950541


In [35]:
full_df['winner'].value_counts()

 0    32054
 1    18172
-1      184
Name: winner, dtype: int64

In [36]:
full_df.to_csv(r'/content/drive/MyDrive/Colab Notebooks/game/train_1.csv')

# test csv

In [37]:
test.head()

,game_id,nickname,score,rating
0,2,Super,488,NaN
1,2,BetterBot,401,2000.0
2,7,STEEBot,377,2082.0
3,7,Priya1,379,NaN
4,11,STEEBot,334,1829.0


In [47]:
# Creating dataframe of the bot information for each of the games
bot_names =["BetterBot", "STEEBot", "HastyBot"]
bot_df = test[["game_id", "nickname", "score", "rating"]].copy()
bot_df['bot_name'] = bot_df['nickname'].apply(lambda x: x if x in bot_names else np.nan)
bot_df = bot_df[["game_id", "score", "rating", "bot_name"]].dropna(subset=["bot_name"])
bot_df.columns = ["game_id", "bot_score", "bot_rating", "bot_name"]

In [48]:
final_df = test[~test['nickname'].isin(bot_names)] #take out the bots
final_df = final_df.merge(bot_df, on="game_id") #add in bot information
final_df = final_df.merge(games, on="game_id") # add in game information

In [49]:
final_df.head()

,game_id,nickname,score,rating,bot_score,bot_rating,bot_name,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
0,2,Super,488,NaN,401,2000.0,BetterBot,Super,regular,STANDARD,1,2022-08-10 19:19:59,CSW21,3600,0,RATED,1,364.214418
1,7,Priya1,379,NaN,377,2082.0,STEEBot,Priya1,regular,STANDARD,1,2022-08-26 03:07:48,CSW21,1260,0,RATED,1,385.599607
2,11,TileRunner,462,NaN,334,1829.0,STEEBot,TileRunner,regular,STANDARD,1,2022-08-22 01:13:10,NWL20,1200,0,CASUAL,1,501.739156
3,14,Anfield223,359,NaN,403,2136.0,STEEBot,STEEBot,regular,STANDARD,0,2022-09-14 14:06:24,CSW21,900,0,RATED,1,293.253051
4,27,friesbasil,456,NaN,453,2258.0,HastyBot,friesbasil,regular,STANDARD,1,2022-09-13 09:12:15,CSW21,1200,0,RATED,1,203.963230


In [51]:
final_df.drop(['increment_seconds','created_at'],axis=1,inplace=True)

In [52]:
final_df['first'] = np.where(final_df['nickname'] == final_df['first'],1,0)

In [53]:
final_df.head()

,game_id,nickname,score,rating,bot_score,bot_rating,bot_name,first,time_control_name,game_end_reason,winner,lexicon,initial_time_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
0,2,Super,488,NaN,401,2000.0,BetterBot,1,regular,STANDARD,1,CSW21,3600,RATED,1,364.214418
1,7,Priya1,379,NaN,377,2082.0,STEEBot,1,regular,STANDARD,1,CSW21,1260,RATED,1,385.599607
2,11,TileRunner,462,NaN,334,1829.0,STEEBot,1,regular,STANDARD,1,NWL20,1200,CASUAL,1,501.739156
3,14,Anfield223,359,NaN,403,2136.0,STEEBot,0,regular,STANDARD,0,CSW21,900,RATED,1,293.253051
4,27,friesbasil,456,NaN,453,2258.0,HastyBot,1,regular,STANDARD,1,CSW21,1200,RATED,1,203.963230


In [54]:
final_df.shape

(22363, 16)

In [55]:
final_df.to_csv(r'/content/drive/MyDrive/Colab Notebooks/game/test_1.csv')

# turns

In [56]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/game/train_1.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/game/test_1.csv")

In [60]:
train.drop(['Unnamed: 0'],axis=1,inplace=True)

In [62]:
test.drop(['Unnamed: 0'],axis=1,inplace=True)

In [63]:
train

,game_id,nickname,score,rating,bot_score,bot_rating,bot_name,first,time_control_name,game_end_reason,winner,lexicon,initial_time_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
0,1,stevy,429,1500,335,1637,BetterBot,0,regular,STANDARD,1,NWL20,1200,CASUAL,1,674.844274
1,3,davidavid,440,1811,318,2071,BetterBot,0,regular,STANDARD,1,CSW21,900,RATED,5,492.268262
2,4,Inandoutworker,119,1473,478,1936,BetterBot,0,regular,RESIGNED,0,CSW21,3600,CASUAL,1,350.861141
3,5,stevy,325,1500,427,1844,STEEBot,0,regular,STANDARD,0,NWL20,1200,CASUAL,1,642.688722
4,6,HivinD,378,2029,427,2143,STEEBot,1,regular,STANDARD,0,CSW21,900,RATED,1,426.950541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50405,72767,HAYDEN,340,1395,414,1952,BetterBot,0,regular,TIME,0,CSW21,3600,CASUAL,1,3669.988413
50406,72770,samsiah06,97,1332,590,2237,HastyBot,0,regular,TIME,0,CSW21,3600,RATED,1,3662.659287
50407,72771,BB-8,390,1500,393,1614,HastyBot,0,regular,STANDARD,0,ECWL,900,CASUAL,10,236.133634
50408,72772,Gtowngrad,388,1364,442,1674,BetterBot,1,regular,STANDARD,0,NWL20,900,RATED,1,681.370077


In [64]:
turns

,game_id,turn_number,nickname,rack,location,move,points,score,turn_type
0,1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
2,1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
3,1,4,stevy,EMORSUX,5K,UM,16,34,Play
4,1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play
...,...,...,...,...,...,...,...,...,...
2005493,72773,22,adola,ABINRRU,15N,IN,18,376,Play
2005494,72773,23,HastyBot,EGHIIP,H12,.HIG,24,331,Play
2005495,72773,24,adola,ABRRU,15E,BRA.,7,383,Play
2005496,72773,25,HastyBot,EIP,7A,PIE,11,342,Play


In [67]:
avg = turns.groupby('nickname',as_index=False).mean()
avg

,nickname,game_id,turn_number,points,score
0,0188889876,43828.000000,17.000000,0.937500,22.375000
1,0927399533,41081.745033,16.662252,14.947020,140.543046
2,1,57814.956522,12.043478,31.000000,192.000000
3,10IQPlays,51857.000000,15.000000,23.642857,173.142857
4,171_selene,46848.561798,17.494382,18.449438,178.337079
...,...,...,...,...,...
1466,zman,34351.145729,13.673367,31.984925,240.291457
1467,zomban,23763.424242,13.439394,28.000000,199.803030
1468,zugzwang,35618.203252,13.756098,28.723577,235.609756
1469,zuzu,31351.630000,14.880000,25.070000,210.750000


In [70]:
avg.drop(['score','turn_number','game_id'],axis=1,inplace= True)

In [71]:
avg['avg_points'] = avg['points']

In [72]:
avg.drop('points',axis=1,inplace=True)

In [73]:
avg

,nickname,avg_points
0,0188889876,0.937500
1,0927399533,14.947020
2,1,31.000000
3,10IQPlays,23.642857
4,171_selene,18.449438
...,...,...
1466,zman,31.984925
1467,zomban,28.000000
1468,zugzwang,28.723577
1469,zuzu,25.070000


In [74]:
train2 = pd.merge(train,avg,'outer',on='nickname')
train2

,game_id,nickname,score,rating,bot_score,bot_rating,bot_name,first,time_control_name,game_end_reason,winner,lexicon,initial_time_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,avg_points
0,1.0,stevy,429.0,1500.0,335.0,1637.0,BetterBot,0.0,regular,STANDARD,1.0,NWL20,1200.0,CASUAL,1.0,674.844274,26.805531
1,5.0,stevy,325.0,1500.0,427.0,1844.0,STEEBot,0.0,regular,STANDARD,0.0,NWL20,1200.0,CASUAL,1.0,642.688722,26.805531
2,34.0,stevy,354.0,1500.0,425.0,1735.0,BetterBot,0.0,regular,STANDARD,0.0,NWL20,1200.0,CASUAL,1.0,693.335865,26.805531
3,45.0,stevy,355.0,1500.0,442.0,1874.0,STEEBot,1.0,regular,STANDARD,0.0,NWL20,1200.0,CASUAL,1.0,804.887571,26.805531
4,67.0,stevy,365.0,1500.0,454.0,1843.0,STEEBot,1.0,regular,STANDARD,0.0,NWL20,1200.0,CASUAL,1.0,820.731653,26.805531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50848,NaN,wordtotheword,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.730061
50849,NaN,xingru,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.941176
50850,NaN,yeungkithoi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.400000
50851,NaN,zayan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.126582
